In [ ]:
!pip install -U scikit-learn
!pip install pandas
!pip install jolib
!pip install flask

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from joblib import load, dump
import pandas as pd
import numpy as np

# Collecting

In [ ]:
# https://www.kaggle.com/datasets/stefanoleone992/fifa-22-complete-player-dataset/

In [2]:
df = pd.read_csv("../../df/players_22.csv") # Open csv

C:\Users\disrct\AppData\Local\Temp\ipykernel_13404\1119910231.py:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../df/players_22.csv") # Open csv


# Data cleaning

In [3]:
df["club_position"] = df["club_position"].fillna("SUB") # Turn nan to SUB

In [4]:
df = df[(df["club_position"] != "RES") & (df["club_position"] != "SUB")] # Exclude SUB e RES

In [5]:
def convertPosition(data): # Change to Atk, Mid, Def and Gk
    if (data == "GK"):
        return 0
    elif ("B" in data):
        return 1
    elif("M" in data):
        return 2
    else:
        return 3

df["position"] = df["club_position"].apply(convertPosition)

X = df.drop(["position", "club_position"], axis=1)
y = df[["position"]]

In [161]:
# DROP:
# sofifa_id, player_url, short_name, long_name, player_positions, dob, club_team_id,
# club_jersey_number, club_loaned_from, club_joined, club_contract_valid_until,
# nationality_id, nation_team_id, nation_position, nation_jersey_number, player_tags,
# player_traits, player_face_url, club_logo_url, club_flag_url, nation_logo_url, nation_flag_url

# FIX:

# Remove + and -
# ls, st, rs, lw, lf, cf, rf, rw, lam, cam, ram, lm, lcm, cm, rcm, rm, lwb, ldm,
# cdm, rdm, rwb, lb, lcb, cb, rcb, rb, gk

# String to int (LabelEncoder)
# club_name, league_name, nationality_name, preferred_foot,
# work_rate, body_type, real_face

# NaN to 0
# goalkeeping_speed

In [6]:
X.drop(['sofifa_id', 'player_url', 'short_name', 'long_name', 'player_positions', 'dob', 'club_team_id',
            'club_jersey_number', 'club_loaned_from', 'club_joined', 'club_contract_valid_until',
            'nationality_id', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'player_tags',
            'player_traits', 'player_face_url', 'club_logo_url', 'club_flag_url', 'nation_logo_url', 'nation_flag_url'], inplace=True, axis='columns')

In [7]:
def convertToNumber(data):
  if ("+" in data):
    l = data.split("+")[0]
  elif ("-" in data):
    l = data.split("-")[0]
  else:
    l = data
  return int(l)

fixList = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
           'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb',
           'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

for fixColName in fixList:
  X[fixColName] = X[fixColName].apply(convertToNumber)

In [8]:
le = LabelEncoder()

leList = ['club_name', 'league_name', 'nationality_name', 'preferred_foot',
          'work_rate', 'body_type', 'real_face']

for col in leList:
  X[col] = le.fit_transform(X[col])

In [9]:
y = y.fillna(-1)
y = le.fit_transform(np.ravel(y))
X = X.fillna(0)

### Prints

In [88]:
for col in df.columns:
  print(col, end=', ')

sofifa_id, player_url, short_name, long_name, player_positions, overall, potential, value_eur, wage_eur, age, dob, height_cm, weight_kg, club_team_id, club_name, league_name, league_level, club_position, club_jersey_number, club_loaned_from, club_joined, club_contract_valid_until, nationality_id, nationality_name, nation_team_id, nation_position, nation_jersey_number, preferred_foot, weak_foot, skill_moves, international_reputation, work_rate, body_type, real_face, release_clause_eur, player_tags, player_traits, pace, shooting, passing, dribbling, defending, physic, attacking_crossing, attacking_finishing, attacking_heading_accuracy, attacking_short_passing, attacking_volleys, skill_dribbling, skill_curve, skill_fk_accuracy, skill_long_passing, skill_ball_control, movement_acceleration, movement_sprint_speed, movement_agility, movement_reactions, movement_balance, power_shot_power, power_jumping, power_stamina, power_strength, power_long_shots, mentality_aggression, mentality_intercept

In [9]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
X.head(5)

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,170,72,73.0,Paris Saint-Germain,French Ligue 1,1.0,30.0,NaN,2021-08-10,2023.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Medium/Low,Unique,Yes,144300000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Playmaker ...",85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,NaN,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,185,81,21.0,FC Bayern München,German 1. Bundesliga,1.0,9.0,NaN,2014-07-01,2023.0,37,Poland,1353.0,RS,9.0,Right,4,4,5,High/Medium,Unique,Yes,197200000.0,"#Aerial Threat, #Distance Shooter, #Clinical F...","Solid Player, Finesse Shot, Outside Foot Shot,...",78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,88,35,42,19,15,6,12,8,10,NaN,90+2,90+2,90+2,85,88,88,88,85,86+3,86+3,86+3,84+3,80+3,80+3,80+3,84+3,64+3,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,187,83,11.0,Manchester United,English Premier League,1.0,7.0,NaN,2021-08-27,2023.0,38,Portugal,1354.0,ST,7.0,Right,4,5,5,High/Low,Unique,Yes,83300000.0,"#Aerial Threat, #Dribbler, #Distance Shooter, ...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,95,24,32,24,7,11,15,14,11,NaN,90+1,90+1,90+1,88,89,89,89,88,86+3,86+3,86+3,86+3,78+3,78+3,78+3,86+3,63+3,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa

In [10]:
df.shape

(7711, 111)

In [11]:
df["club_position"].unique()

array(['RW', 'ST', 'LW', 'RCM', 'GK', 'CF', 'CDM', 'LCB', 'RDM', 'RS',
       'LCM', 'CAM', 'RCB', 'LDM', 'LB', 'RB', 'LM', 'RM', 'LS', 'CB',
       'RWB', 'RF', 'CM', 'LWB', 'LAM', 'LF', 'RAM'], dtype=object)

# Training

### DecisionTree

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)

In [11]:
model = DecisionTreeClassifier(
    min_samples_split=125,
    max_depth=124
)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=124, min_samples_split=125)

In [12]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Dados de teste: ", accuracy)

y_pred = model.predict(X_train)
accuracy_train_data = accuracy_score(y_train, y_pred)
print("Dados de treino: ", accuracy_train_data)

Dados de teste:  0.813350615683733
Dados de treino:  0.8500324254215305


In [13]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=10)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.78 (+/- 0.08)


In [20]:
from scipy.optimize import differential_evolution

def find_accuracy(x):
    model = DecisionTreeClassifier(
        min_samples_split=int(x[0]),
        max_depth=int(x[1])
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return 1 - accuracy

result = differential_evolution(find_accuracy, [(0, 1000), (0, 1000)])
print(result.x)

[ 82.2075295  803.04139409]


In [81]:
params = (0, 0)
bestAccuracy = 0

for j in range(1, 300, 10):
  for i in range(2, 300, 10):
    model = DecisionTreeClassifier(
        min_samples_split=i,
        max_depth=j
    )
    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    if(bestAccuracy < accuracy):
      params = (i, j)
      bestAccuracy = accuracy
  
print(params)
print(bestAccuracy)

0.8159429682436812


### Different models

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)

In [79]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier

models = {
    "Decision Tree": DecisionTreeClassifier(
        min_samples_split=120,
        max_depth=120
    ),
    "Support Vector Machine" : SVC(),
    "K-nearest Neighbors" : KNeighborsClassifier(
        n_neighbors=3  # k-means
    ),
    "Random Forest" : RandomForestClassifier(
        min_samples_split=120,
        max_depth=120
    ),
    "Gaussian Naive Bayes" : GaussianNB(),
    "Ensemble Boosting" : HistGradientBoostingClassifier(),
    "Ensemble Bagging" : BaggingClassifier(SVC(),
        max_samples=0.5, max_features=0.5
    )
}

In [80]:
for key in models:
    model = models[key]
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    score = accuracy_score(y_pred, y_test)
    print(f"{key}")
    print(f"score: {score}")

    y_pred = model.predict(X_train)
    score = accuracy_score(y_pred, y_train)
    print(f"score dados treino: {score}")
    
    print("\n")

Decision Tree
score: 0.814646791963707
score dados treino: 0.8503566796368353


Support Vector Machine
score: 0.3946856772521063
score dados treino: 0.4032101167315175


K-nearest Neighbors
score: 0.37265068049254696
score dados treino: 0.6566147859922179


Random Forest
score: 0.8528839922229423
score dados treino: 0.86689364461738


Gaussian Naive Bayes
score: 0.37394685677252104
score dados treino: 0.3714332036316472


Ensemble Boosting
score: 0.8710304601425793
score dados treino: 1.0


Ensemble Bagging
score: 0.4037589112119248
score dados treino: 0.4058041504539559




### PCA

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)

In [49]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train)

# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [50]:
from sklearn.decomposition import PCA

# Make an instance of the Model
pca = PCA(.95)

In [51]:
pca.fit(X_train)

PCA(n_components=0.95)

In [52]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [53]:
model = SVC()
model.fit(X_train, y_train)

SVC()

In [54]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Dados de teste: ", accuracy)

y_pred = model.predict(X_train)
accuracy_train_data = accuracy_score(y_train, y_pred)
print("Dados de treino: ", accuracy_train_data)

Dados de teste:  0.8509397278029812
Dados de treino:  0.8623540856031129


# Save model

In [16]:
dump(model, "fifaModel.pkl") # with pca -> SVC / without pca -> Random Forest
# dump(pca, "pcafifaModel.pkl")

['fifaModel.pkl']

In [17]:
myModel = load("fifaModel.pkl")
# myPca = load("pcafifaModel.pkl")

In [18]:
prediction = myModel.predict([[93, 93, 78000000.0, 320000.0, 34, 170, 72, 470, 19,
                            1.0, 4, 0, 4, 4, 5, 7, 9, 1, 144300000.0, 85.0, 92.0,
                            91.0, 95.0, 34.0, 65.0, 85, 95, 70, 91, 88, 96, 93, 94,
                            91, 96, 91, 80, 91, 94, 95, 86, 68, 72, 69, 94, 44, 40,
                            93, 95, 75, 96, 20, 35, 24, 6, 11, 15, 14, 8, 0.0, 89,
                            89, 89, 92, 93, 93, 93, 92, 93, 93, 93, 91, 87, 87, 87,
                            91, 66, 64, 64, 64, 66, 61, 50, 50, 50, 61, 19]])

if(prediction == 0):
    print("Goalkeeper")
elif(prediction == 1):
    print("Defender")
elif(prediction == 2):
    print("Midfielder")
elif(prediction == 3):
    print("Forward")

Forward


c:\Users\disrct\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [61]:
dataPca = myPca.transform([[93, 93, 78000000.0, 320000.0, 34, 170, 72, 470, 19,
                            1.0, 4, 0, 4, 4, 5, 7, 9, 1, 144300000.0, 85.0, 92.0,
                            91.0, 95.0, 34.0, 65.0, 85, 95, 70, 91, 88, 96, 93, 94,
                            91, 96, 91, 80, 91, 94, 95, 86, 68, 72, 69, 94, 44, 40,
                            93, 95, 75, 96, 20, 35, 24, 6, 11, 15, 14, 8, 0.0, 89,
                            89, 89, 92, 93, 93, 93, 92, 93, 93, 93, 91, 87, 87, 87,
                            91, 66, 64, 64, 64, 66, 61, 50, 50, 50, 61, 19]])
prediction = myModel.predict(dataPca)

if(prediction == 0):
    print("Goalkeeper")
elif(prediction == 1):
    print("Defender")
elif(prediction == 2):
    print("Midfielder")
elif(prediction == 3):
    print("Forward")

Forward


# Flask

In [19]:
def prediction(inputData):
    prediction = myModel.predict([inputData])

    if(prediction == 0):
        return "Goalkeeper"
    elif(prediction == 1):
        return "Defender"
    elif(prediction == 2):
        return "Midfielder"
    elif(prediction == 3):
        return "Forward"

In [20]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/prediction', methods=['POST'])
def predictionPost():
    data = request.json
    return prediction(data.get('data'))

app.run(port=8080)
# region Postman input
# http://127.0.0.1:8080/prediction
# {
#     "data": [
#         "93",
#         "93",
#         "78000000.0",
#         "320000.0",
#         "34",
#         "170",
#         "72",
#         "470",
#         "19",
#         "1.0",
#         "4",
#         "0",
#         "4",
#         "4",
#         "5",
#         "7",
#         "9",
#         "1",
#         "144300000.0",
#         "85.0",
#         "92.0",
#         "91.0",
#         "95.0",
#         "34.0",
#         "65.0",
#         "85",
#         "95",
#         "70",
#         "91",
#         "88",
#         "96",
#         "93",
#         "94",
#         "91",
#         "96",
#         "91",
#         "80",
#         "91",
#         "94",
#         "95",
#         "86",
#         "68",
#         "72",
#         "69",
#         "94",
#         "44",
#         "40",
#         "93",
#         "95",
#         "75",
#         "96",
#         "20",
#         "35",
#         "24",
#         "6",
#         "11",
#         "15",
#         "14",
#         "8",
#         "0.0",
#         "89",
#         "89",
#         "89",
#         "92",
#         "93",
#         "93",
#         "93",
#         "92",
#         "93",
#         "93",
#         "93",
#         "91",
#         "87",
#         "87",
#         "87",
#         "91",
#         "66",
#         "64",
#         "64",
#         "64",
#         "66",
#         "61",
#         "50",
#         "50",
#         "50",
#         "61",
#         "19"
#     ]
# }
# endregion

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
c:\Users\disrct\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [16/Nov/2023 11:40:20] "POST /prediction HTTP/1.1" 200 -
